## LAB 4 notbook
## LLM testing

In [1]:
from poly_llm.to_test.file_name_check import file_name_check
from poly_llm.common.abstract_executor import AbstractExecutor
from poly_llm.common.prompt_generator import PromptGenerator
from poly_llm.generators.llm_test_generator import LLMTestGenerator
from transformers import AutoTokenizer, T5ForConditionalGeneration
import json

import importlib

c:\Users\gumen\anaconda3\envs\lab4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Measuring the code coverage

In [2]:
executor = AbstractExecutor(file_name_check)

inputs = [
    "example.txt",
    "1example.dll",
    's1sdf3.asd',
    'K.dll',
    'MY16FILE3.exe',
    'His12FILE94.exe',
    '_Y.txt',
    '?aREYA.exe',
    '/this_is_valid.dll',
    'this_is_valid.wow',
]

# Execute the inputs and print the results
for input in inputs:
    #exceptions, execution_time, coverage = executor._execute_input(input)
    coverage_date = executor._execute_input(input)
    print(coverage_date)

Program module: file_name_check
Lines: ['def file_name_check(file_name):\n', '    """Create a function which takes a string representing a file\'s name, and returns\n', "    'Yes' if the the file's name is valid, and returns 'No' otherwise.\n", "    A file's name is considered to be valid if and only if all the following conditions \n", '    are met:\n', "    - There should not be more than three digits ('0'-'9') in the file's name.\n", "    - The file's name contains exactly one dot '.'\n", '    - The substring before the dot should not be empty, and it starts with a letter from \n', "    the latin alphapet ('a'-'z' and 'A'-'Z').\n", "    - The substring after the dot should be one of these: ['txt', 'exe', 'dll']\n", '    Examples:\n', '    file_name_check("example.txt") # => \'Yes\'\n', '    file_name_check("1example.dll") # => \'No\' (the name should start with a latin alphapet letter)\n', '    """\n', "    suf = ['txt', 'exe', 'dll']\n", "    lst = file_name.split(sep='.')\n", '   

### Running the promt with the LLM

In [4]:
executor = AbstractExecutor(file_name_check)
prompt_generator = PromptGenerator(file_name_check)

model_name = "Salesforce/codet5-large-ntp-py"
tokenizer = AutoTokenizer.from_pretrained(model_name) #tokenizer#AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Python-hf")#
model = T5ForConditionalGeneration.from_pretrained(model_name) 

llm_generator = LLMTestGenerator(model, tokenizer, file_name_check)
prompt = prompt_generator.generate_prompt(few_shot_examples=['''def test_file_name_check(): \n 
assert file_name_check("example.txt") == 'Yes'\n
assert file_name_check("1example.dll") == 'No' \n'''])

print(f"THE PROMPT {prompt}")
test, test_name = llm_generator.create_test_function(prompt)


Program module: file_name_check
Lines: ['def file_name_check(file_name):\n', '    """Create a function which takes a string representing a file\'s name, and returns\n', "    'Yes' if the the file's name is valid, and returns 'No' otherwise.\n", "    A file's name is considered to be valid if and only if all the following conditions \n", '    are met:\n', "    - There should not be more than three digits ('0'-'9') in the file's name.\n", "    - The file's name contains exactly one dot '.'\n", '    - The substring before the dot should not be empty, and it starts with a letter from \n', "    the latin alphapet ('a'-'z' and 'A'-'Z').\n", "    - The substring after the dot should be one of these: ['txt', 'exe', 'dll']\n", '    Examples:\n', '    file_name_check("example.txt") # => \'Yes\'\n', '    file_name_check("1example.dll") # => \'No\' (the name should start with a latin alphapet letter)\n', '    """\n', "    suf = ['txt', 'exe', 'dll']\n", "    lst = file_name.split(sep='.')\n", '   

### Measuring the coverage achived by the LLM produced code

In [6]:
filename = "test_generated.py"
llm_generator.write_test_to_file(test, filename=filename)

module_name = filename.split(".")[0]
function_name = test_name

# Dynamically import the module
module = importlib.import_module(module_name)
function = getattr(module, function_name)

executor2 = AbstractExecutor(function)

coverage_data = executor2._execute_input(file_name_check)
print(coverage_data)

Test function written to test_generated.py
Program module: test_file_name_check
Lines: ['def test_file_name_check(file_name_check):\n', '    assert file_name_check("example.txt") == \'Yes\'\n', '    assert file_name_check("1example.dll") == \'No\'\n', '    assert file_name_check("example.txt") == \'Yes\'\n', '    assert file_name_check("1example.dll") == \'No\'\n', '    assert file_name_check("example.txt") == \'Yes\'\n', '    assert file_name_check("1example.dll") == \'No\'\n']
{'input': <function file_name_check at 0x000001EB65AC28B0>, 'exceptions': 0, 'execution_time': 0.0010199546813964844, 'coverage': {'covered_lines': 16, 'num_statements': 22, 'percent_covered': 70.58823529411765, 'percent_covered_display': '71', 'missing_lines': 6, 'excluded_lines': 5, 'num_branches': 12, 'num_partial_branches': 4, 'covered_branches': 8, 'missing_branches': 4}}
